# HPO Two-Stage System - Job Submission & Monitoring

This notebook integrates the newly created **two-stage HPO system** with SLURM job management.

## Overview
- **Stage 1**: Optimize model architecture for MAE (50 trials, ~4-10 hours)
- **Stage 2**: Calibrate quantile levels for PICP ≈ 80% (20 trials, ~2-3 hours)
- **Resources**: A100 GPU, 10-hour time limit, 32GB RAM
- **Models**: NHITS_Q and TIMESNET_Q on heat and water datasets

## Workflow
1. Submit all Stage 1 experiments (priority order)
2. Monitor progress
3. Submit Stage 2 as Stage 1 completes
4. View and compare results

## 1. Imports and Setup

Import required libraries and verify the HPO infrastructure is in place.

In [14]:
import os
import sys
import json
import subprocess
from datetime import datetime
import pandas as pd

# Detect project root by searching upwards or for known repo folder name
def find_repo_root(start_dir):
    path = os.path.abspath(start_dir)
    # Upward search for markers
    for _ in range(6):
        if os.path.exists(os.path.join(path, 'hpo')) and os.path.exists(os.path.join(path, 'README.md')):
            return path
        parent = os.path.abspath(os.path.join(path, '..'))
        if parent == path:
            break
        path = parent
    # Fallback: look for known repo folder in current directory
    try:
        for name in os.listdir(start_dir):
            candidate = os.path.join(start_dir, name)
            if name == 'ExAI-Timeseries-Thesis' and os.path.isdir(candidate):
                return candidate
    except Exception:
        pass
    return os.path.abspath(start_dir)

project_root = find_repo_root(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Change to project root for all operations
os.chdir(project_root)

print(f'✓ Project root: {project_root}')
print(f'✓ Current directory: {os.getcwd()}')
print(f'✓ Python version: {sys.version}')
print(f'✓ Imports ready')

✓ Project root: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis
✓ Current directory: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis
✓ Python version: 3.9.23 | packaged by conda-forge | (main, Jun  4 2025, 17:57:12) 
[GCC 13.3.0]
✓ Imports ready


## 2. HPO Configuration - A100 GPU with 10 Hours

Define SLURM and HPO settings for submitting jobs.

In [15]:
# ============================================================================
# HPO RUNNER CONFIGURATION
# ============================================================================

# SLURM Resource Configuration (informational)
SLURM_CONFIG = {
    'partition': 'a100',              # Valid GPU partition
    'gres': 'gpu:a100:1',             # A100 GPU (1 GPU)
    'time': '10:00:00',               # 10 hours
    'cpus_per_task': 8,               # 8 CPU cores
    'job_name_prefix': 'hpo',         # Job name prefix
}

# HPO Experiment Configuration
HPO_CONFIG = {
    'stage1_trials': 50,              # Stage 1: 50 trials
    'stage2_trials': 20,              # Stage 2: 20 trials
    'priority_order': [
        ('water', 'TIMESNET_Q'),      # Best overall performer
        ('heat', 'NHITS_Q'),          # Best heat, worst PICP
        ('water', 'NHITS_Q'),         # Fast training
        ('heat', 'TIMESNET_Q'),       # Architecture comparison
    ],
}

# Paths (absolute)
TRACKING_FILE = os.path.join(project_root, 'hpo/tracking/experiments.csv')
JOB_MAP_FILE = os.path.join(project_root, 'hpo/hpo_current_jobs.json')

# Scripts (absolute)
SUBMIT_SCRIPT = os.path.join(project_root, 'hpo/submit_experiment.py')
CHECK_STATUS_SCRIPT = os.path.join(project_root, 'hpo/check_status.py')

print('✓ SLURM Config:', SLURM_CONFIG)
print('✓ HPO Config - Stage1 trials:', HPO_CONFIG['stage1_trials'])
print('✓ HPO Config - Stage2 trials:', HPO_CONFIG['stage2_trials'])
print('✓ Priority order:', HPO_CONFIG['priority_order'])
print('✓ Using scripts:')
print('  - submit:', SUBMIT_SCRIPT)
print('  - status:', CHECK_STATUS_SCRIPT)
print('✓ Job map:', JOB_MAP_FILE)

✓ SLURM Config: {'partition': 'a100', 'gres': 'gpu:a100:1', 'time': '10:00:00', 'cpus_per_task': 8, 'job_name_prefix': 'hpo'}
✓ HPO Config - Stage1 trials: 50
✓ HPO Config - Stage2 trials: 20
✓ Priority order: [('water', 'TIMESNET_Q'), ('heat', 'NHITS_Q'), ('water', 'NHITS_Q'), ('heat', 'TIMESNET_Q')]
✓ Using scripts:
  - submit: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/submit_experiment.py
  - status: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/check_status.py
✓ Job map: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/hpo_current_jobs.json


## 3. Utility Functions

Helper functions to submit jobs, check status, and manage the HPO workflow.

In [16]:
def submit_stage_job(stage, model, dataset, trials, verbose=True):
    """
    Submit a single HPO stage job using submit_experiment.py
    
    Args:
        stage: 1 or 2
        model: 'NHITS_Q' or 'TIMESNET_Q'
        dataset: 'heat' or 'water'
        trials: number of trials
        verbose: print output
    
    Returns:
        job_id or None if submission failed
    """
    cmd = [
        'python', SUBMIT_SCRIPT,
        '--stage', str(stage),
        '--model', model,
        '--dataset', dataset,
        '--trials', str(trials)
    ]
    
    if verbose:
        print(f'Submitting Stage {stage}: {dataset.upper()} + {model}')
        print(f'  Script: {SUBMIT_SCRIPT}')
        print(f'  CWD: {project_root}')
        print(f'  Command: {" ".join(cmd)}')
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=60, cwd=project_root)
        if result.returncode == 0:
            # Parse job_id from output
            output_lines = result.stdout.strip().split('\n')
            for line in output_lines:
                # Look for "Job ID: <number>" pattern
                if 'Job ID:' in line:
                    parts = line.split(':')
                    if len(parts) >= 2:
                        job_id = parts[-1].strip()
                        if job_id and job_id.isdigit():
                            if verbose:
                                print(f'  ✓ Submitted as Job {job_id}')
                            return job_id
            # Fallback: show stdout and return success token
            if verbose and result.stdout:
                print(result.stdout)
            print(f'  ✓ Submission successful (no Job ID parsed)')
            return 'submitted'
        else:
            if verbose and result.stdout:
                print(result.stdout)
            print(f'  ✗ Submission failed:')
            print(f'    stderr: {result.stderr}')
            return None
    except subprocess.TimeoutExpired:
        print(f'  ✗ Submission timed out')
        return None
    except Exception as e:
        print(f'  ✗ Error: {e}')
        return None


def check_experiment_status(model=None, dataset=None, verbose=True):
    """
    Check status of all or filtered experiments using check_status.py
    
    Args:
        model: Filter by model ('NHITS_Q', 'TIMESNET_Q', or None for all)
        dataset: Filter by dataset ('heat', 'water', or None for all)
        verbose: print output
    
    Returns:
        Status output text or None if check failed
    """
    cmd = ['python', CHECK_STATUS_SCRIPT]
    
    if model:
        cmd.extend(['--model', model])
    if dataset:
        cmd.extend(['--dataset', dataset])
    
    if verbose:
        filter_str = f'{dataset}+{model}' if dataset and model else (dataset or model or 'all')
        print(f'Checking status ({filter_str})...')
        print(f'  Script: {CHECK_STATUS_SCRIPT}')
        print(f'  CWD: {project_root}\n')
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=60, cwd=project_root)
        if result.returncode == 0:
            if verbose:
                print(result.stdout)
            return result.stdout
        else:
            print(f'Error checking status:')
            print(result.stderr)
            return None
    except Exception as e:
        print(f'Error: {e}')
        return None

print('✓ Utility functions defined: submit_stage_job(), check_experiment_status()')

✓ Utility functions defined: submit_stage_job(), check_experiment_status()


## 4. SUBMIT STAGE 1 BY PRIORITY - Select Single Job

Submit one Stage 1 experiment by selecting a priority number (1-4).

In [9]:
print('='*80)
print('STAGE 1 - SELECT PRIORITY TO SUBMIT')
print('='*80 + '\n')

print('Available priorities:\n')
for i, (dataset, model) in enumerate(HPO_CONFIG['priority_order'], 1):
    print(f'  {i}. {dataset.upper():6} + {model}')

print()
priority = int(input('Enter priority number (1-4): '))

if 1 <= priority <= 4:
    dataset, model = HPO_CONFIG['priority_order'][priority - 1]
    
    print(f'\n' + '='*80)
    print(f'Submitting Priority {priority}: {dataset.upper()} + {model}')
    print('='*80 + '\n')
    
    job_id = submit_stage_job(
        stage=1,
        model=model,
        dataset=dataset,
        trials=HPO_CONFIG['stage1_trials'],
        verbose=True
    )
    
    if job_id:
        exp_name = f'{dataset}_{model.lower()}'
        job_entry = {exp_name: {'job_id': job_id, 'stage': 1, 'status': 'submitted', 'priority': priority}}
        
        # Save to job map
        if os.path.exists(JOB_MAP_FILE):
            with open(JOB_MAP_FILE, 'r') as f:
                existing = json.load(f)
            existing.update(job_entry)
        else:
            existing = job_entry
        
        with open(JOB_MAP_FILE, 'w') as f:
            json.dump(existing, f, indent=2)
        
        print(f'\n✓ Job submitted successfully')
        print(f'  Priority: {priority}')
        print(f'  Dataset: {dataset.upper()}')
        print(f'  Model: {model}')
        print(f'  Job ID: {job_id}')
        print(f'  Saved to: {JOB_MAP_FILE}')
    else:
        print('✗ Submission failed')
else:
    print('Invalid priority number. Please enter 1-4.')

STAGE 1 - SELECT PRIORITY TO SUBMIT

Available priorities:

  1. WATER  + TIMESNET_Q
  2. HEAT   + NHITS_Q
  3. WATER  + NHITS_Q
  4. HEAT   + TIMESNET_Q


Submitting Priority 4: HEAT + TIMESNET_Q

Submitting Stage 1: HEAT + TIMESNET_Q
  Script: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/submit_experiment.py
  CWD: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis
  Command: python /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/submit_experiment.py --stage 1 --model TIMESNET_Q --dataset heat --trials 50
  ✓ Submitted as Job 1475318

✓ Job submitted successfully
  Priority: 4
  Dataset: HEAT
  Model: TIMESNET_Q
  Job ID: 1475318
  Saved to: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/hpo_current_jobs.json


## 4. SUBMIT STAGE 1 - All Models (Priority Order)

Submit all Stage 1 architecture optimization experiments.

**Timeline**: ~4-10 hours per model on A100 (depends on model and dataset)
**Trials**: 50 per model
**Output**: best_params.json for each model+dataset combination

In [4]:
print('='*80)
print('SUBMITTING STAGE 1 - ARCHITECTURE OPTIMIZATION')
print('='*80)
print(f'\nPriority order:')
for i, (dataset, model) in enumerate(HPO_CONFIG['priority_order'], 1):
    print(f'  {i}. {dataset.upper():6} + {model}')

print(f'\nConfirm submission:')
response = input('  Submit all Stage 1 jobs? (yes/no): ').strip().lower()

if response in ['yes', 'y']:
    print('\n' + '='*80)
    stage1_jobs = {}
    
    for dataset, model in HPO_CONFIG['priority_order']:
        job_id = submit_stage_job(
            stage=1,
            model=model,
            dataset=dataset,
            trials=HPO_CONFIG['stage1_trials'],
            verbose=True
        )
        if job_id:
            exp_name = f'{dataset}_{model.lower()}'
            stage1_jobs[exp_name] = {'job_id': job_id, 'stage': 1, 'status': 'submitted'}
            print()
    
    print('='*80)
    print(f'✓ Submitted {len(stage1_jobs)} Stage 1 experiments')
    print('='*80)
    
    # Save job map
    if stage1_jobs:
        if os.path.exists(JOB_MAP_FILE):
            with open(JOB_MAP_FILE, 'r') as f:
                existing = json.load(f)
            existing.update(stage1_jobs)
        else:
            existing = stage1_jobs
        
        with open(JOB_MAP_FILE, 'w') as f:
            json.dump(existing, f, indent=2)
        print(f'Saved job map to {JOB_MAP_FILE}')
else:
    print('Submission cancelled.')

SUBMITTING STAGE 1 - ARCHITECTURE OPTIMIZATION

Priority order:
  1. WATER  + TIMESNET_Q
  2. HEAT   + NHITS_Q
  3. WATER  + NHITS_Q
  4. HEAT   + TIMESNET_Q

Confirm submission:
Submission cancelled.


## 5. CHECK STAGE 1 STATUS

Monitor the progress of Stage 1 experiments. Run this cell periodically to check completion.

In [4]:
print('='*80)
print('STAGE 1 STATUS CHECK')
print('='*80 + '\n')

check_experiment_status(verbose=True)

STAGE 1 STATUS CHECK

Checking status (all)...
  Script: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/check_status.py
  CWD: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis



HPO EXPERIMENT STATUS

STAGE 1: ARCHITECTURE
--------------------------------------------------------------------------------

? WATER - TIMESNET_Q
  Job ID: 1475012
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:11:34
  MAE: 0.002022

? HEAT - NHITS_Q
  Job ID: 1475013
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:12:41
  MAE: 0.026382

? WATER - NHITS_Q
  Job ID: 1475014
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:13:08
  MAE: 0.072588

? HEAT - TIMESNET_Q
  Job ID: 1475015
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:13:29
  MAE: 0.089717

? WATER - TIMESNET_Q
  Job ID: 1475317
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-28 15:32:21
  MAE: 0.002022

? HEAT - TIMESNET_Q
  Job ID: 1475318
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-28 15:32:30
  MAE: 0.089717

STAGE 2: CALIBRATION
--------------------------------------------------------------------------------

? WAT

'================================================================================\nHPO EXPERIMENT STATUS\n================================================================================\n\nSTAGE 1: ARCHITECTURE\n--------------------------------------------------------------------------------\n\n? WATER - TIMESNET_Q\n  Job ID: 1475012\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:11:34\n  MAE: 0.002022\n\n? HEAT - NHITS_Q\n  Job ID: 1475013\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:12:41\n  MAE: 0.026382\n\n? WATER - NHITS_Q\n  Job ID: 1475014\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:13:08\n  MAE: 0.072588\n\n? HEAT - TIMESNET_Q\n  Job ID: 1475015\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:13:29\n  MAE: 0.089717\n\n? WATER - TIMESNET_Q\n  Job ID: 1475317\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-28 15:32:21\n  MAE: 0.002022\n\n? HEAT - TIMESNET_Q\n  Job ID: 

## 6. SUBMIT STAGE 2 BY PRIORITY - Select Single Job

Submit one Stage 2 quantile calibration experiment by selecting a priority number (1-4).

**Prerequisite**: Corresponding Stage 1 job must be complete.

In [12]:
print('='*80)
print('STAGE 2 - SELECT PRIORITY TO SUBMIT')
print('='*80 + '\n')

print('Available priorities:\n')
for i, (dataset, model) in enumerate(HPO_CONFIG['priority_order'], 1):
    print(f'  {i}. {dataset.upper():6} + {model}')

print()
priority = int(input('Enter priority number (1-4): '))

if 1 <= priority <= 4:
    dataset, model = HPO_CONFIG['priority_order'][priority - 1]
    
    print(f'\n' + '='*80)
    print(f'Submitting Stage 2 - Priority {priority}: {dataset.upper()} + {model}')
    print('='*80 + '\n')
    
    job_id = submit_stage_job(
        stage=2,
        model=model,
        dataset=dataset,
        trials=HPO_CONFIG['stage2_trials'],
        verbose=True
    )
    
    if job_id:
        exp_name = f'{dataset}_{model.lower()}_s2'
        job_entry = {exp_name: {'job_id': job_id, 'stage': 2, 'status': 'submitted', 'priority': priority}}
        
        # Save to job map
        if os.path.exists(JOB_MAP_FILE):
            with open(JOB_MAP_FILE, 'r') as f:
                existing = json.load(f)
            existing.update(job_entry)
        else:
            existing = job_entry
        
        with open(JOB_MAP_FILE, 'w') as f:
            json.dump(existing, f, indent=2)
        
        print(f'\n✓ Stage 2 job submitted successfully')
        print(f'  Priority: {priority}')
        print(f'  Dataset: {dataset.upper()}')
        print(f'  Model: {model}')
        print(f'  Job ID: {job_id}')
        print(f'  Saved to: {JOB_MAP_FILE}')
    else:
        print('✗ Submission failed (Stage 1 may not be complete)')
else:
    print('Invalid priority number. Please enter 1-4.')

STAGE 2 - SELECT PRIORITY TO SUBMIT

Available priorities:

  1. WATER  + TIMESNET_Q
  2. HEAT   + NHITS_Q
  3. WATER  + NHITS_Q
  4. HEAT   + TIMESNET_Q


Submitting Stage 2 - Priority 4: HEAT + TIMESNET_Q

Submitting Stage 2: HEAT + TIMESNET_Q
  Script: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/submit_experiment.py
  CWD: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis
  Command: python /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/submit_experiment.py --stage 2 --model TIMESNET_Q --dataset heat --trials 20
  ✓ Submitted as Job 1475599

✓ Stage 2 job submitted successfully
  Priority: 4
  Dataset: HEAT
  Model: TIMESNET_Q
  Job ID: 1475599
  Saved to: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/hpo_current_jobs.json


## 7. SUBMIT STAGE 2 - All Models (Priority Order)

After Stage 1 completes, submit all Stage 2 experiments to calibrate quantiles for PICP ≈ 80%.

**Prerequisite**: Stage 1 must complete first (automatic check in submit script)
**Timeline**: ~2-3 hours per model on A100
**Trials**: 20 per model
**Output**: calibrated_quantiles.json for each model+dataset combination

In [5]:
print('='*80)
print('SUBMITTING STAGE 2 - QUANTILE CALIBRATION')
print('='*80)
print(f'\nStage 2 will calibrate quantiles for each Stage 1 result')
print(f'Same priority order:\n')

for i, (dataset, model) in enumerate(HPO_CONFIG['priority_order'], 1):
    print(f'  {i}. {dataset.upper():6} + {model}')

print(f'\nNote: Stage 2 automatically checks that Stage 1 completed first.')
print(f'Confirm submission:')
response = input('  Submit all Stage 2 jobs? (yes/no): ').strip().lower()

if response in ['yes', 'y']:
    print('\n' + '='*80)
    stage2_jobs = {}
    
    for dataset, model in HPO_CONFIG['priority_order']:
        job_id = submit_stage_job(
            stage=2,
            model=model,
            dataset=dataset,
            trials=HPO_CONFIG['stage2_trials'],
            verbose=True
        )
        if job_id:
            exp_name = f'{dataset}_{model.lower()}_s2'
            stage2_jobs[exp_name] = {'job_id': job_id, 'stage': 2, 'status': 'submitted'}
            print()
        else:
            print(f'  (Skipping - Stage 1 likely not complete)\n')
    
    print('='*80)
    print(f'✓ Submitted {len(stage2_jobs)} Stage 2 experiments')
    print('='*80)
    
    # Save job map
    if stage2_jobs:
        if os.path.exists(JOB_MAP_FILE):
            with open(JOB_MAP_FILE, 'r') as f:
                existing = json.load(f)
            existing.update(stage2_jobs)
        else:
            existing = stage2_jobs
        
        with open(JOB_MAP_FILE, 'w') as f:
            json.dump(existing, f, indent=2)
        print(f'Updated job map: {JOB_MAP_FILE}')
else:
    print('Submission cancelled.')

SUBMITTING STAGE 2 - QUANTILE CALIBRATION

Stage 2 will calibrate quantiles for each Stage 1 result
Same priority order:

  1. WATER  + TIMESNET_Q
  2. HEAT   + NHITS_Q
  3. WATER  + NHITS_Q
  4. HEAT   + TIMESNET_Q

Note: Stage 2 automatically checks that Stage 1 completed first.
Confirm submission:
Submission cancelled.


## 8. CHECK STAGE 2 STATUS

Monitor Stage 2 experiments as they progress toward PICP calibration.

In [18]:
print('='*80)
print('STAGE 2 STATUS CHECK')
print('='*80 + '\n')

check_experiment_status(verbose=True)

STAGE 2 STATUS CHECK

Checking status (all)...
  Script: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis/hpo/check_status.py
  CWD: /home/hpc/iwi5/iwi5389h/ExAI-Timeseries-Thesis

HPO EXPERIMENT STATUS

STAGE 1: ARCHITECTURE
--------------------------------------------------------------------------------

? WATER - TIMESNET_Q
  Job ID: 1475012
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:11:34
  MAE: 0.002022

? HEAT - NHITS_Q
  Job ID: 1475013
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:12:41
  MAE: 0.026382

? WATER - NHITS_Q
  Job ID: 1475014
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:13:08
  MAE: 0.072588

? HEAT - TIMESNET_Q
  Job ID: 1475015
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-27 17:13:29
  MAE: 0.089717

? WATER - TIMESNET_Q
  Job ID: 1475317
  Status: COMPLETED/FAILED
  Trials: 50
  Submitted: 2025-12-28 15:32:21
  MAE: 0.002022

? HEAT - TIMESNET_Q
  Job ID: 1475318
  Status: COMPLETED

'================================================================================\nHPO EXPERIMENT STATUS\n================================================================================\n\nSTAGE 1: ARCHITECTURE\n--------------------------------------------------------------------------------\n\n? WATER - TIMESNET_Q\n  Job ID: 1475012\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:11:34\n  MAE: 0.002022\n\n? HEAT - NHITS_Q\n  Job ID: 1475013\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:12:41\n  MAE: 0.026382\n\n? WATER - NHITS_Q\n  Job ID: 1475014\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:13:08\n  MAE: 0.072588\n\n? HEAT - TIMESNET_Q\n  Job ID: 1475015\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-27 17:13:29\n  MAE: 0.089717\n\n? WATER - TIMESNET_Q\n  Job ID: 1475317\n  Status: COMPLETED/FAILED\n  Trials: 50\n  Submitted: 2025-12-28 15:32:21\n  MAE: 0.002022\n\n? HEAT - TIMESNET_Q\n  Job ID: 

## 9. VIEW RESULTS - Stage 1 & Stage 2 Outputs

Display and compare optimized hyperparameters and calibrated quantiles.

In [19]:
print('='*80)
print('RESULTS - STAGE 1 ARCHITECTURE OPTIMIZATION')
print('='*80 + '\n')

# Stage 1 results
stage1_results = {}
for dataset, model in HPO_CONFIG['priority_order']:
    exp_name = f'{dataset}_{model.lower()}'
    results_file = f'hpo/results/stage1/{exp_name}/best_params.json'
    
    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            data = json.load(f)
        stage1_results[exp_name] = {
            'MAE': f"{data.get('best_mae', 'N/A'):.6f}",
            'Trials': data.get('n_trials', 'N/A'),
            'Dataset': dataset.upper(),
            'Model': model,
            'LR': f"{data.get('best_params', {}).get('lr', 0):.6f}",
            'Dropout': f"{data.get('best_params', {}).get('dropout', 0):.4f}",
        }
        print(f'✓ {exp_name}: MAE={data.get("best_mae", "N/A"):.6f}, Trials={data.get("n_trials", "N/A")}')
    else:
        print(f'  {exp_name}: Not found')

if stage1_results:
    df1 = pd.DataFrame.from_dict(stage1_results, orient='index')
    print('\n' + '='*80)
    print('Stage 1 Summary:')
    print(df1.to_string())

print('\n' + '='*80)
print('RESULTS - STAGE 2 QUANTILE CALIBRATION')
print('='*80 + '\n')

# Stage 2 results
stage2_results = {}
for dataset, model in HPO_CONFIG['priority_order']:
    exp_name = f'{dataset}_{model.lower()}'
    results_file = f'hpo/results/stage2/{exp_name}/calibrated_quantiles.json'
    
    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            data = json.load(f)
        quantiles = data.get('calibrated_quantiles', [0, 0.5, 0])
        stage2_results[exp_name] = {
            'PICP': f"{data.get('achieved_picp', 0):.1f}%",
            'Target': '80%',
            'Error': f"{data.get('calibration_error', 999):.1f}%",
            'Q_low': f"{quantiles[0]:.4f}" if len(quantiles) > 0 else 'N/A',
            'Q_high': f"{quantiles[2]:.4f}" if len(quantiles) > 2 else 'N/A',
        }
        print(f'✓ {exp_name}: PICP={data.get("achieved_picp", 0):.1f}%, Quantiles=[{quantiles[0]:.4f}, 0.5, {quantiles[2]:.4f}]')
    else:
        print(f'  {exp_name}: Not found (run Stage 2)')

if stage2_results:
    df2 = pd.DataFrame.from_dict(stage2_results, orient='index')
    print('\n' + '='*80)
    print('Stage 2 Summary:')
    print(df2.to_string())

RESULTS - STAGE 1 ARCHITECTURE OPTIMIZATION

✓ water_timesnet_q: MAE=0.002022, Trials=50
✓ heat_nhits_q: MAE=0.026382, Trials=50
✓ water_nhits_q: MAE=0.072588, Trials=53
✓ heat_timesnet_q: MAE=0.089717, Trials=51

Stage 1 Summary:
                       MAE  Trials Dataset       Model        LR Dropout
water_timesnet_q  0.002022      50   WATER  TIMESNET_Q  0.000538  0.0312
heat_nhits_q      0.026382      50    HEAT     NHITS_Q  0.000054  0.0040
water_nhits_q     0.072588      53   WATER     NHITS_Q  0.000015  0.4396
heat_timesnet_q   0.089717      51    HEAT  TIMESNET_Q  0.003582  0.3182

RESULTS - STAGE 2 QUANTILE CALIBRATION

✓ water_timesnet_q: PICP=0.0%, Quantiles=[0.4747, 0.5, 0.5253]
✓ heat_nhits_q: PICP=0.0%, Quantiles=[0.3924, 0.5, 0.6076]
✓ water_nhits_q: PICP=0.0%, Quantiles=[0.3724, 0.5, 0.6276]
✓ heat_timesnet_q: PICP=0.0%, Quantiles=[0.3818, 0.5, 0.6182]

Stage 2 Summary:
                  PICP Target  Error   Q_low  Q_high
water_timesnet_q  0.0%    80%  80.0%  0.4747  0.